# Insertion of HemA and HemL

To enable our organism to make heme from L-glumamate (rather than just glycine), we add new reactions to the model to simulate the heterologous expression of the genes for HemA and HemL, which are found in plants and bacteria.

In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [2]:
model = read_sbml_model('model_standard_yeast.xml')
model.add_boundary(model.metabolites.pheme_m, type='demand')

Reaction identifier,DM_pheme_m
Name,Protoheme C34H30FeN4O4 demand
Memory address,0x01f75d6965b0
Stoichiometry,pheme_m --> Protoheme C34H30FeN4O4 -->
GPR,
Lower bound,0
Upper bound,1000.0


In [3]:
model_base = model.copy()

The "last" metabolite made by the native yeast metabolism, which will be our starting point for the HemA reaction is L-Glutamyl-tRNA(Glu) (id: glutrna_c). This metabolite is made from L-glutamate.

In [4]:
model.metabolites.get_by_id('glutrna_c')

Metabolite identifier,glutrna_c
Name,L-Glutamyl-tRNA(Glu)
Memory address,0x01f76992f4c0
Formula,C5H7NO3R
Compartment,c
In 1 reaction(s),GLUTRS


Metabolite where the plant heme biosynthesis re-connects with the native yeast metabolism is 5-Amino-4-oxopentanoate / 5-Aminolevulinate (id: 5aop_c). 

**Problem**: Not sure if HemA and HemL would work in the cytoplasm or the mitochondrion of the yeast (normally it is produced by ALAS in the mitochondrion). According to this source (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1203387/) "Cell fractionation results indicate that GTR and GSAT are located only in the chloroplasts." Since yeast doesn't have chloroplasts and the bacterium must produce it in the cytoplasm, we have chosen to simply express in the cytoplasm - also to avoid transport of L-Glutamyl-tRNA(Glu) into the mitochondrion. 

In [5]:
model.metabolites.get_by_id('5aop_c')

Metabolite identifier,5aop_c
Name,5-Amino-4-oxopentanoate
Memory address,0x01f7697a5f70
Formula,C5H9NO3
Compartment,c
In 3 reaction(s),"5AOPt2, 5AOPtm, PPBNGS"


The two reactions we are interested in adding are:

HemA (glutamyl-tRNA reductase, https://www.kegg.jp/entry/1.2.1.70): 
L-glutamyl-tRNAGlu + NADPH + H+ -> L-glutamate 1-semialdehyde + NADP+ + tRNAGlu

HemL (glutamate-1-semialdehyde aminotransferase, https://www.kegg.jp/entry/5.4.3.8): L-glutamate 1-semialdehyde -> 5-aminolevulinate

First the new intermediate formed by HemA (and consumed by HemL) is defined (L-glutamate 1-semialdehyde):

In [6]:
L_glu_1_semi = Metabolite(id='L_glu_1_semi_c', compartment='c')

The HemA reaction is defined:

In [7]:
HemA = Reaction('HemA')

In [8]:
HemA.add_metabolites({model.metabolites.h_c: -1,
                      model.metabolites.nadph_c: -1,
                      model.metabolites.glutrna_c: -1,
                      L_glu_1_semi: 1,
                      model.metabolites.nadp_c: 1,
                      model.metabolites.trnaglu_c: 1,
                             })

In [9]:
print(HemA.build_reaction_string())

glutrna_c + h_c + nadph_c --> L_glu_1_semi_c + nadp_c + trnaglu_c


HemL is defined:

In [10]:
HemL = Reaction('HemL')

In [11]:
HemL.add_metabolites({L_glu_1_semi: -1,
                      model.metabolites.get_by_id('5aop_c'): 1,
                             })

In [12]:
print(HemL.build_reaction_string())

L_glu_1_semi_c --> 5aop_c


Both reactions are added to the model:

In [13]:
model.add_reactions([HemA, HemL])

In [14]:
import escher

The new pathway is visualized in red below in addition to the native heme synthesis pathway.

In [15]:
escher.Builder(model = model,
    map_json='IMM904.HemAHemL.json',
              reaction_data={'HemA': 10, 'HemL': 10})

Builder(reaction_data={'HemA': 10, 'HemL': 10})

## Investigation of the effect of inserting HemA and HemL

In [33]:
# Mutant with HemA and HemL:
with model:
    model.objective = model.reactions.FCLTm
    model.optimize()
    print('Parameters for the HemA and HemL strain:')
    print("Rate of heme production:", model.reactions.FCLTm.flux)
    print("Growth rate:", model.reactions.BIOMASS_SC5_notrace.flux)
    print("Yield of heme production:", model.reactions.FCLTm.flux / (-1*model.reactions.EX_glc__D_e.flux))

Parameters for the HemA and HemL strain:
Rate of heme production: 0.5501893939393939
Growth rate: 0.0
Yield of heme production: 0.05501893939393939


In [34]:
# Non-mutant yeast
with model_base:
    model_base.objective = model_base.reactions.FCLTm
    model_base.optimize()
    print('Parameters for the WT strain without HemA and HemL:')
    print("Rate of heme production:", model_base.reactions.FCLTm.flux)
    print("Growth rate:", model_base.reactions.BIOMASS_SC5_notrace.flux)
    print("Yield of heme production:", model_base.reactions.FCLTm.flux / (-1*model.reactions.EX_glc__D_e.flux))

Parameters for the WT strain without HemA and HemL:
Rate of heme production: 0.5501893939393958
Growth rate: -2.1743679008842116e-17
Yield of heme production: 0.05501893939393958


From the above, it looks like the HemA and HemL genes does not give the organism an added advantage under normal conditions when compared to the WT.
However, if the glycine mitchondrial transporter is knocked out, it is clear that the glumatamate can substitute as a substrate for heme production:

In [36]:
# Mutant with HemA and HemL:
with model as GLYt2m_mutant:
    GLYt2m_mutant.reactions.get_by_id('GLYt2m').bounds = 0, 0
    GLYt2m_mutant.objective = THRA_mutant.reactions.FCLTm
    GLYt2m_mutant.optimize()
    print('Parameters for the HemA and HemL strain:')
    print("Rate of heme production:", GLYt2m_mutant.reactions.FCLTm.flux)
    print("Growth rate:", GLYt2m_mutant.reactions.BIOMASS_SC5_notrace.flux)

Parameters for the HemA and HemL strain:
Rate of heme production: 0.44989775051124803
Growth rate: 0.0


In [35]:
# Non-mutant yeast
with model_base as GLYt2m_mutant:
    GLYt2m_mutant.reactions.get_by_id('GLYt2m').bounds = 0, 0
    GLYt2m_mutant.objective = THRA_mutant.reactions.FCLTm
    GLYt2m_mutant.optimize()
    print('Parameters for the WT strain without HemA and HemL:')
    print("Rate of heme production:", GLYt2m_mutant.reactions.FCLTm.flux)
    print("Growth rate:", GLYt2m_mutant.reactions.BIOMASS_SC5_notrace.flux)

Parameters for the WT strain without HemA and HemL:
Rate of heme production: -2.196419827518402e-14
Growth rate: 0.10634914877962603
